# Estimating Queue Priority Wait Times

https://dl.acm.org/doi/10.1145/3427921.3450249

In [1]:
import pandas
from scipy.optimize import newton

In [2]:
P_GROUP_BOUNDS = [1,3,10]

COLS_TO_USE = [4,5,6,9,10,12,13,14,15,16]
HEADER_NAMES = ['size','weight','recieved_time','fee','confirmed_block_height','confirm_time','waiting_time','fee_rate','enter_block_height','no_block_confirm']

### Data Loading

In [3]:
d1 = pandas.read_csv('TimetxinBlock621500.csv', usecols=COLS_TO_USE, names=HEADER_NAMES)
d2 = pandas.read_csv('TimetxinBlock622000.csv', usecols=COLS_TO_USE, names=HEADER_NAMES)
d3 = pandas.read_csv('TimetxinBlock622500.csv', usecols=COLS_TO_USE, names=HEADER_NAMES)
dataFrame = pandas.concat([d1, d2, d3])

dataFrame.head(10)

,size,weight,recieved_time,fee,confirmed_block_height,confirm_time,waiting_time,fee_rate,enter_block_height,no_block_confirm
0,223,892,1583793568,2486,621001,1583794214,646,11.147982,621000,2
1,223,892,1583791819,2486,621001,1583794214,2395,11.147982,620998,4
2,223,892,1583793885,2486,621001,1583794214,329,11.147982,621001,1
3,223,892,1583793700,2486,621001,1583794214,514,11.147982,621000,2
4,223,892,1583791354,2486,621001,1583794214,2860,11.147982,620998,4
5,223,892,1583791731,2486,621001,1583794214,2483,11.147982,620998,4
6,371,1484,1583792024,4136,621001,1583794214,2190,11.148248,620998,4
7,371,1484,1583793603,4136,621001,1583794214,611,11.148248,621000,2
8,371,1484,1583793883,4136,621001,1583794214,331,11.148248,621001,1
9,371,1484,1583791921,4136,621001,1583794214,2293,11.148248,620998,4


### Block Size

In [4]:
block_groups = dataFrame.groupby(['confirmed_block_height'])['confirmed_block_height'].count()
mean_block_size = float(round(block_groups.mean()))
print("Mean Block Size: ", mean_block_size)

Mean Block Size:  2266.0


### Arrival Rates (Lambda) - Priority

In [5]:
p_groups_lambda = []

transaction_count = len(dataFrame)
block_confirm_max = dataFrame['no_block_confirm'].max()
sum_lambda = 0.0
for i in range(len(P_GROUP_BOUNDS)+1):
    p_transaction_count = 0.0
    p_lambda = 0.0
    if(i == 0):
        p_df = dataFrame[dataFrame['no_block_confirm'].between(0, P_GROUP_BOUNDS[i])]
        p_transaction_count = len(p_df)
        p_lambda = float(p_transaction_count) / (float(p_df['recieved_time'].max()) - float(p_df['recieved_time'].min()))
    elif(i == (len(P_GROUP_BOUNDS))):
        p_df = dataFrame[dataFrame['no_block_confirm'].between((P_GROUP_BOUNDS[i-1]+1), block_confirm_max)]
        p_transaction_count = len(p_df)
        p_lambda = float(p_transaction_count) / (float(p_df['recieved_time'].max()) - float(p_df['recieved_time'].min()))
    else:
        p_df = dataFrame[dataFrame['no_block_confirm'].between((P_GROUP_BOUNDS[i-1]+1), P_GROUP_BOUNDS[i])]
        p_transaction_count = len(p_df)
        p_lambda = float(p_transaction_count) / (float(p_df['recieved_time'].max()) - float(p_df['recieved_time'].min()))

    p_groups_lambda.append(p_lambda)
    sum_lambda += p_lambda
    print("Priority Group " + str(i+1))
    print("Transaction Count: " + str(p_transaction_count))
    print("Lambda: " + str(p_lambda) + "\n")

print("Sum Of Lambda: ", sum_lambda)

Priority Group 1
Transaction Count: 2035992
Lambda: 1.9164135440894505

Priority Group 2
Transaction Count: 565752
Lambda: 0.5319602867074807

Priority Group 3
Transaction Count: 388749
Lambda: 0.36553531432826897

Priority Group 4
Transaction Count: 399103
Lambda: 0.3759231995900763

Sum Of Lambda:  3.1898323447152768


### Service Rate (Mu)

In [15]:
service_time = 600
mu = 1/service_time
#mu = (1/service_time)*mean_block_size
#mu = mean_block_size / service_time
print("Mu: ", mu)

Mu:  0.0016666666666666668


### Validation Of Z Using Newton Method (Mine Vs SciPy)

In [16]:
my_p_group_z = []
sciPy_p_group_z = []
lambda_sum = 0.0

def NewtonMethod(lam, m_u, block_size, x0, epsilon):
    fx = lambda x: lam*(1 - x) - m_u*x*(1 - x**block_size)
    dfx = lambda x: m_u*(block_size*x**block_size + x**block_size - 1) - lam
    while True:
        x1 = x0 - fx(x0) / dfx(x0)
        t = abs(x1 - x0)
        if t < epsilon:
            break
        x0 = x1
    return x0

def SciPyNewton(lam, m_u, block_size, x0, epsilon, max_iteration):
    fx = lambda x: lam*(1 - x) - m_u*x*(1 - x**block_size)
    #dfx = lambda x: m_u*(block_size*x**block_size + x**block_size - 1) - lam
    return newton(fx, x0, fprime=None, args=(), tol=epsilon, maxiter=max_iteration, fprime2=None)

for i in range(len(p_groups_lambda)):
    lambda_sum += p_groups_lambda[i]
    my_p_group_z.append(NewtonMethod(lambda_sum, mu, mean_block_size, 0, 1e-10))
    sciPy_p_group_z.append(SciPyNewton(lambda_sum, mu, mean_block_size, 0, 1e-10, 500))

for i in range(len(sciPy_p_group_z)):
    print("Priority Group " + str(i+1))
    print("My Z: \n" + str(my_p_group_z[i]))
    print("SciPy Z: \n" + str(sciPy_p_group_z[i]) + "\n")

Priority Group 1
My Z: 
0.9993147806098512
SciPy Z: 
0.9993147806474754

Priority Group 2
My Z: 
0.9995855087963951
SciPy Z: 
0.9995855087969598

Priority Group 3
My Z: 
0.9997263332086644
SciPy Z: 
0.9997263332086649

Priority Group 4
My Z: 
0.9998465892552025
SciPy Z: 
0.999846589295516



### Wait Times Using Z

In [17]:
prev_transaction_in_queue = 0.0
p_group_response_time = []

for i in range(len(p_groups_lambda)):
    transaction_in_queue = my_p_group_z[i] / (1 - my_p_group_z[i]) - prev_transaction_in_queue
    response_time  = transaction_in_queue / p_groups_lambda[i]

    prev_transaction_in_queue = transaction_in_queue

    p_group_response_time.append(response_time)

    print("Priority Group " + str(i+1))
    print("Z: ", my_p_group_z[i])
    print("Average Transaction In Queue (L): ", transaction_in_queue)
    print("Average Response Time In Queue (W): " + str(response_time) + "\n")

Priority Group 1
Z:  0.9993147806098512
Average Transaction In Queue (L):  1458.3866057743926
Average Response Time In Queue (W): 760.9978599203225

Priority Group 2
Z:  0.9995855087963951
Average Transaction In Queue (L):  953.2098288489833
Average Response Time In Queue (W): 1791.881560837535

Priority Group 3
Z:  0.9997263332086644
Average Transaction In Queue (L):  2699.8688963034715
Average Response Time In Queue (W): 7386.068569776693

Priority Group 4
Z:  0.9998465892552025
Average Transaction In Queue (L):  3817.579347463318
Average Response Time In Queue (W): 10155.210829302845

